<a href="https://colab.research.google.com/github/sanam077/code/blob/master/CONTRASTIVE_WORDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DATA

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

In [3]:
def find_all_occurence_and_replace(definition_pairs, max_examples = 5000, max_length = 512, min_length = 8):

    dataset = tfds.load('multi_news', split="train", shuffle_files=False)
    definition_sentences = []
    rng = np.random.default_rng()
    for definition_words in definition_pairs:
        bias_sentence_pair1 = []
        bias_sentence_pair2 = []
        for newsarticle in dataset:
            newsarticle = str(newsarticle["document"])
            for sentence in newsarticle.split("."):
              if len(sentence)>min_length & len(sentence)<max_length:
                  for i,equalword in enumerate(definition_words):
                    for a,word in enumerate(equalword):
                        if word in sentence:
                            listwithout_word = definition_words[:i] +definition_words[i+1:]
                            replace_word = rng.choice(listwithout_word, axis = 0)[a]
                            bias_sentence_pair1.append(sentence.replace(word,replace_word))
                            bias_sentence_pair2.append(sentence)
#                             print(sentence.replace(word,replace_word))
#                             print(sentence)
            if ((max_examples != 0) & (len(bias_sentence_pair1)>max_examples)):
                break
        print("found ",len(bias_sentence_pair1), "of word", definition_words)
        definition_sentences.append([bias_sentence_pair1,bias_sentence_pair2])
    return definition_sentences


def get_understanding_set(definition_pairs,tokenizer,max_examples = 20000):
        definiton_train = find_all_occurence_and_replace(definition_pairs, max_examples = max_examples)
        tokenized_definition_train=[]
        for definition_set in definiton_train:
            new_pair = []
            for pair in definition_set:
                new_def_set_pair = tokenizer(pair, max_length=128, padding=True, truncation=True, return_tensors='tf')
                new_pair.append(new_def_set_pair)
            tokenized_definition_train.append(new_pair)
        return tokenized_definition_train



def load_pretrain_ds( batch_size = 8, max_length = 1024, min_length = 10):
    dataset = tfds.load('multi_news', split="train", shuffle_files=False)
    allsentences = []
    for newsarticle in dataset:
        text = str(newsarticle["document"])
        for sentence in text.split("."):
            if len(sentence)>min_length & len(sentence)<max_length:
                allsentences.append(sentence)
        if len(allsentences) > 1000000:
            break
    new_ds = []
    new_batch = []
    for sentence in allsentences:
        if len(sentence)>min_length:
            new_batch.append(sentence)
            if len(new_batch)>= batch_size:
                new_ds.append(new_batch)
                new_batch = []
#	if len(new_ds) > 1000000:
#		break
    return new_ds


**MODEL**

In [10]:
!pip install transformers
from transformers import TFBertModel,TFBertForMaskedLM, BertTokenizer
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time


class Understandable_Embedder(tf.keras.Model):

    def __init__(self, batch_size = 8, target_units=768, train_only_dense=False, contrastive_scale = 0.01, debias_freq = 10):
      super(Understandable_Embedder, self).__init__()
      self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states=True)
      self.batch_size = batch_size
      self.bert = TFBertForMaskedLM.from_pretrained('bert-base-uncased')
      self.dropout = layers.Dropout(0.2)
      self.train_only_dense = train_only_dense
      if self.train_only_dense:
          self.dense_projection = layers.Dense(units = target_units)
      self.dense = layers.Dense(units = 2, activation = "softmax")
    #  self.dense_headless = layers.Dense(units = target_units, activation = "Relu")
      self.compare_loss = keras.losses.MeanAbsoluteError()
      self.contrastive_scale = tf.constant(contrastive_scale)
      self.debiasing_freq = debias_freq
      self.mask_token = "[MASK]"
      self.masked_id = 103






    def __call__(self, inputs, training = False):
      x = self.bert.bert(inputs,training=training).last_hidden_state[:,0]
     # print(x)
    #  x = self.dense_headless(x)
      x = self.dropout(x,training=training)
      if self.train_only_dense:
          x = self.dense_projection(x)
      outputs = self.dense(x)
      return outputs

    def call_pre_training(self, inputs, training = True):
        outputs = self.bert.call(inputs,training=training)
        return outputs

    def call_headless(self, inputs, training = False):
      x = self.bert.bert(inputs,training=training).last_hidden_state[:,0]
      if self.train_only_dense:
          x = self.dense_projection(x)
      x = self.dropout(x,training=training)
   #   x = self.dense_headless(x)
      return x


    # def eval_simple_project(self,dataset,P, dataset_length):
    #     # accuracy = 0
    #     # step = 0
    #     # for x,y in dataset:
    #     #     step = step + self.batch_size
    #     #     if step > dataset_length:
    #     #         break
    #     #     x = self.call_headless(x,training=False)
    #     #     x = np.matmul(P,x.numpy().transpose()).transpose()
    #     #     y_pred = self.dense(tf.convert_to_tensor(x))
    #     #     for i,pred in enumerate(y_pred):
    #     #             if np.argmax(y_pred[i].numpy()) == y[i].numpy():
    #     #                 accuracy = accuracy +1.0
    #     # accuracy = accuracy /dataset_length


    #     accuracy = 0.0
    #     step = 0

    #     for x,y in dataset:
    #         step = step + self.batch_size
    #         if step > dataset_length :
    #             break
    #         # x = self.call_headless(x,training=False)
    #         # x = np.matmul(P,x.numpy().transpose()).transpose()
    #         # y_pred = self.dense(tf.convert_to_tensor(x))
    #       #  print(y_pred)
    #         y_pred = self(x,training = False)
    #         # print(y_pred)
    #         for i,pred in enumerate(y):
    #             if np.argmax(y_pred[i].numpy()) == pred.numpy():
    #                 accuracy = accuracy +1.0
    #     accuracy = accuracy /step
    #     return accuracy
    def eval_simple_project(self, dataset, batch_size, dataset_length, P, verbose = False):
        accuracy = 0.0
        step = 0

        for x,y in dataset:
            step = step + batch_size
            if step > dataset_length :
                break
            x = self.call_headless(x,training=False)
            x = np.matmul(P,x.numpy().transpose()).transpose()
            y_pred = self.dense(tf.convert_to_tensor(x))
            #y_pred = self(x,training=False)
           # print(y_pred)
            if verbose:
                if step % np.round(dataset_length/5) < batch_size:
                    print("at", step, "of", dataset_length)
            for i,pred in enumerate(y):
                if np.argmax(y_pred[i].numpy()) == pred.numpy():
                    accuracy = accuracy +1.0
        accuracy = accuracy /step
        return accuracy

    def evaluate(self, dataset, batch_size, dataset_length, verbose = False):
        accuracy = 0.0
        step = 0

        for x,y in dataset:
            step = step + batch_size
            if step > dataset_length :
                break
            y_pred = self(x,training=False)
           # print(y_pred)
            if verbose:
                if step % np.round(dataset_length/5) < batch_size:
                    print("at", step, "of", dataset_length)
            for i,pred in enumerate(y):
                if np.argmax(y_pred[i].numpy()) == pred.numpy():
                    accuracy = accuracy +1.0
        accuracy = accuracy /step
        return accuracy

    def predict_simple(self,inputs):
        if len(inputs) > self.batch_size:
            start = True
            x = np.asarray([])
            import math
            for i in range(math.ceil(len(inputs)/self.batch_size)):
                new_inputs = inputs[i*self.batch_size:min((i+1)*self.batch_size, len(inputs))]
                tokenized_inputs = self.tokenizer(new_inputs, max_length=128, padding=True, truncation=True, return_tensors='tf')
                if start:
                    start = False
                    x = self.call_headless(tokenized_inputs,training=False)
                else:
                    x = np.concatenate((x,self.call_headless(tokenized_inputs,training=False)))

            return x
        else:
            tokenized_inputs = self.tokenizer(inputs, max_length=128, padding=True, truncation=True, return_tensors='tf')
            x = self.call_headless(tokenized_inputs,training=False)
            return x.numpy()


    @tf.function
    def normal_train_step(self,x,y):
      with tf.GradientTape() as tape:
          y_pred = self(x, training=True)  # Forward pass
          # Compute our own loss
          loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

      # Compute gradients
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)

      # Update weights
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))

      # Compute our own metrics
      self.compiled_metrics.update_state(y, y_pred)
      return loss



    @tf.function
    def delete_dim(self,i,vec, axis = 1):
        split1,split2,split3 = tf.split(vec,[i,1,(vec.shape[axis]-(i+1))],axis)
        short_vec = tf.concat([split1,split3],1)
        return short_vec

    @tf.function
    def compare_train_step(self,x1,x2,i,loss_factor):
        with tf.GradientTape() as tape:

            y_pred_1 = self.call_headless(x1, training=True)  # Forward pass
            y_pred_2 = self.call_headless(x2, training=True)

            #contrastive_loss = (1.0/self.compare_loss(y_pred_1[:,i],y_pred_2[:,i]))* self.contrastive_scale

#             if contrastive_loss < self.contrastive_scale*self.batch_size*2.0:
#                 contrastive_loss = 0.0

#             y_pred_1 = self.delete_dim(i,y_pred_1)
#             y_pred_2 = self.delete_dim(i,y_pred_2)

            loss = self.compare_loss(y_pred_1,y_pred_2)*loss_factor #+ contrastive_loss*20.0)r #scale loss by 1/number of set meaning dimensions
       #   tf.multiply(loss[0,i], 0)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        return loss



    def fit_classify_understandable(self, dataset,definition_pairs,epochs,steps_per_epoch , report_intervall = 20, loss_factor = 1.0):
        self.bert.mlm.trainable = False
        self.bert.bert.trainable = not self.train_only_dense
        history = {}
        history["loss"] = []
        history["loss_compare"] = []
        for e in range(epochs):
            print("At epoch", e+1, "of", epochs)
            step = 0
            loss_report = 0.0
            loss_report_compare = 0.0
            for x,y in dataset:

                if step > steps_per_epoch :
                    break
                loss = self.normal_train_step(x,y)
                loss_report = loss_report + +float(loss)

                if step % self.debiasing_freq == 0:
                    self.dense.trainable = False
                    for current_attribute_id,attribute_set in enumerate(definition_pairs):
                        start = ((e*steps_per_epoch +step)*self.batch_size) % len(attribute_set)
                        stop = start + self.batch_size

                        #one dictionary for each word
                        feed_dict_1 = {}
                        feed_dict_1["input_ids"] = attribute_set[0]["input_ids"][start:stop,:]
                        feed_dict_1["token_type_ids"] = attribute_set[0]["token_type_ids"][start:stop,:]
                        feed_dict_1["attention_mask"] = attribute_set[0]["attention_mask"][start:stop,:]

                        #one dictionary for each word
                        feed_dict_2 = {}
                        feed_dict_2["input_ids"] = attribute_set[1]["input_ids"][start:stop,:]
                        feed_dict_2["token_type_ids"] = attribute_set[1]["token_type_ids"][start:stop,:]
                        feed_dict_2["attention_mask"] = attribute_set[1]["attention_mask"][start:stop,:]

                       # print(feed_dict_1)
                        loss_compare = self.compare_train_step(feed_dict_1,feed_dict_2,tf.constant(current_attribute_id),tf.constant(loss_factor * 1.0/len(definition_pairs)))
                        loss_report_compare = loss_report_compare + float(loss_compare)
                    self.dense.trainable = True

                step = step +1
                if step % report_intervall == 0:
                    print("Average Training loss at step",step, "/", steps_per_epoch,":", loss_report/report_intervall)
                    print("Average Training loss_compare at step",step, "/", steps_per_epoch,":", loss_report_compare/report_intervall)
                    history["loss"].append(loss_report/report_intervall)
                    history["loss_compare"].append(loss_report_compare/report_intervall)
                    loss_report = 0.0
                    loss_report_compare = 0.0

        return history

    def fit_understandable(self,definition_pairs,epochs,steps_per_epoch , report_intervall = 20):
        self.bert.mlm.trainable = False
        self.dense.trainable = False
        self.bert.bert.trainable = True
        history = {}
        history["loss_compare"] = []
        for e in range(epochs):
            print("At epoch", e+1, "of", epochs)
            loss_report_compare = 0.0
            for step in range(steps_per_epoch):

                for current_attribute_id,attribute_set in enumerate(definition_pairs):
                    start = ((e*steps_per_epoch +step)*self.batch_size) % len(attribute_set)
                    stop = start + self.batch_size

                    #one dictionary for each word
                    feed_dict_1 = {}
                    feed_dict_1["input_ids"] = attribute_set[0]["input_ids"][start:stop,:]
                    feed_dict_1["token_type_ids"] = attribute_set[0]["token_type_ids"][start:stop,:]
                    feed_dict_1["attention_mask"] = attribute_set[0]["attention_mask"][start:stop,:]

                    #one dictionary for each word
                    feed_dict_2 = {}
                    feed_dict_2["input_ids"] = attribute_set[1]["input_ids"][start:stop,:]
                    feed_dict_2["token_type_ids"] = attribute_set[1]["token_type_ids"][start:stop,:]
                    feed_dict_2["attention_mask"] = attribute_set[1]["attention_mask"][start:stop,:]

                   # print(feed_dict_1)
                    loss_compare = self.compare_train_step(feed_dict_1,feed_dict_2,tf.constant(current_attribute_id),tf.constant(1.0/len(definition_pairs)))
                    loss_report_compare = loss_report_compare + float(loss_compare)

                if step % report_intervall == 0:
                    print("Average Training loss_compare at step",step, "/", steps_per_epoch,":", loss_report_compare/report_intervall)
                    history["loss_compare"].append(loss_report_compare/report_intervall)
                    loss_report_compare = 0.0

        return history

    def fit_classify(self, dataset,epochs,steps_per_epoch , report_intervall = 20):
        self.bert.mlm.trainable = False
        self.bert.bert.trainable = not self.train_only_dense
        history = {}
        history["loss"] = []
        history["loss_compare"] = []
        for e in range(epochs):
            print("At epoch", e+1, "of", epochs)
            step = 0
            avg_loss = 0.0
            avg_loss_compare = 0.0
            loss_report = 0.0
            for x,y in dataset:

                if step > steps_per_epoch :
                    break
                loss = self.normal_train_step(x,y)
                avg_loss = avg_loss+float(loss)
                loss_report = loss_report + +float(loss)

                step = step +1
                if step % report_intervall == 0:
                    print("Average Training loss at step",step, "/", steps_per_epoch,":", loss_report/report_intervall)
                    history["loss"].append(loss_report/report_intervall)
                    loss_report = 0.0

        return history



#     def compute_loss(self, labels: tf.Tensor, logits: tf.Tensor) -> tf.Tensor:
#         loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
#             from_logits=True, reduction=tf.keras.losses.Reduction.NONE
#         )
#         # make sure only labels that are not equal to -100
#         # are taken into account as loss
#         masked_lm_active_loss = tf.not_equal(tf.reshape(tensor=labels["labels"], shape=(-1,)), -100)
#         masked_lm_reduced_logits = tf.boolean_mask(
#             tensor=tf.reshape(tensor=logits[0], shape=(-1, shape_list(logits[0])[2])),
#             mask=masked_lm_active_loss,
#         )
#         masked_lm_labels = tf.boolean_mask(
#             tensor=tf.reshape(tensor=labels["labels"], shape=(-1,)), mask=masked_lm_active_loss
#         )
#         next_sentence_active_loss = tf.not_equal(tf.reshape(tensor=labels["next_sentence_label"], shape=(-1,)), -100)
#         next_sentence_reduced_logits = tf.boolean_mask(
#             tensor=tf.reshape(tensor=logits[1], shape=(-1, 2)), mask=next_sentence_active_loss
#         )
#         next_sentence_label = tf.boolean_mask(
#             tensor=tf.reshape(tensor=labels["next_sentence_label"], shape=(-1,)), mask=next_sentence_active_loss
#         )
#         masked_lm_loss = loss_fn(y_true=masked_lm_labels, y_pred=masked_lm_reduced_logits)
#         next_sentence_loss = loss_fn(y_true=next_sentence_label, y_pred=next_sentence_reduced_logits)
#         masked_lm_loss = tf.reshape(tensor=masked_lm_loss, shape=(-1, shape_list(next_sentence_loss)[0]))
#         masked_lm_loss = tf.reduce_mean(input_tensor=masked_lm_loss, axis=0)
#
#         return masked_lm_loss + next_sentence_loss


   # @tf.function
    def pretrain_train_step(self,data):
      with tf.GradientTape() as tape:
          trainingoutput = self.call_pre_training(data, training=True)  # Forward pass
          # Compute our own loss
          loss = trainingoutput[0]

      # Compute gradients
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)

      # Update weights
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))

      # Compute our own metrics
    #  self.compiled_metrics.update_state(y, y_pred)
      return tf.math.reduce_mean(loss)


    def mask(self, batch):


        inputs =  self.tokenizer(batch, max_length=128, padding=True, truncation=True, return_tensors = "tf")

#         import time
#         start = time.time()
        new_input_ids = inputs["input_ids"].numpy()
        inputs["labels"] = tf.identity(inputs["input_ids"])

        for x,a in enumerate(new_input_ids):
           for y,_ in enumerate(a):
               if not new_input_ids[x,y] == 0:
                   if np.random.random() < 0.15 :
                       new_input_ids[x,y] = self.masked_id

        inputs["input_ids"] = tf.convert_to_tensor(new_input_ids, dtype = tf.int32)

#         end = time.time()
#         print("this took:", end-start)  ## not very efficient but still only takes 0.005 sec per batch
        return inputs

    def secondsToText(self,secs):
        days = secs//86400
        hours = (secs - days*86400)//3600
        minutes = (secs - days*86400 - hours*3600)//60
        seconds = secs - days*86400 - hours*3600 - minutes*60
        result = ("{} days, ".format(days) if days else "") + \
        ("{} hours, ".format(hours) if hours else "") + \
        ("{} minutes, ".format(minutes) if minutes else "") + \
        ("{} seconds, ".format(seconds) if seconds else "")
        return result

    def fit_pretrain(self, dataset,definition_pairs,epochs,steps_per_epoch,tokenizer):
        self.tokenizer = tokenizer
        self.dense.trainable = False
      #  print(self.bert.bert.layers)
        self.bert.bert.pooler.trainable = False

        losses ={}
        losses["compare"] =[]
        losses["mlm"] =[]
        for e in range(epochs):
            starttime = time.time()
            print("At epoch", e+1, "of", epochs)
            step = 0
            avg_loss = 0.0
            avg_loss_compare = 0.0
            for batch in dataset:
                if step > steps_per_epoch :
                    break
                loss = self.pretrain_train_step(self.mask(batch))
                losses["mlm"].append(float(loss))
                avg_loss = avg_loss+float(loss)
                if step % self.debiasing_freq == 0:
                    self.bert.mlm.trainable = False
                    for current_attribute_id,attribute_set in enumerate(definition_pairs):
                        start = ((e*steps_per_epoch +step)*self.batch_size) % len(attribute_set)
                        stop = start + self.batch_size

                        #one dictionary for each word
                        feed_dict_1 = {}
                        feed_dict_1["input_ids"] = attribute_set[0]["input_ids"][start:stop,:]
                        feed_dict_1["token_type_ids"] = attribute_set[0]["token_type_ids"][start:stop,:]
                        feed_dict_1["attention_mask"] = attribute_set[0]["attention_mask"][start:stop,:]

                        #one dictionary for each word
                        feed_dict_2 = {}
                        feed_dict_2["input_ids"] = attribute_set[1]["input_ids"][start:stop,:]
                        feed_dict_2["token_type_ids"] = attribute_set[1]["token_type_ids"][start:stop,:]
                        feed_dict_2["attention_mask"] = attribute_set[1]["attention_mask"][start:stop,:]

                       # print(feed_dict_1)
                        loss = self.compare_train_step(feed_dict_1,feed_dict_2,tf.constant(current_attribute_id),tf.constant( 1.0/len(definition_pairs)))
                        losses["compare"].append(float(loss))
                        avg_loss_compare = avg_loss_compare+float(loss)
                    self.bert.mlm.trainable = True

                step = step +1

                if step % 20 == 0:
                    print("Training loss at step",step, "/", steps_per_epoch,":", avg_loss/step ," loss_compare:", avg_loss_compare/step)
                    elapsed = time.time() - starttime
                    print("average time per step ",self.secondsToText(elapsed/step),"predicted time until completion of epoch:", self.secondsToText((elapsed/step)*(steps_per_epoch-step) ))
        return losses


**FINE_TUNE**

In [16]:
!pip install model
!pip install Understandable_Embedder
!pip install data import find_all_occurence_and_replace,load_pretrain_ds,get_understanding_set
import os
import pickle
from transformers import DataCollatorForLanguageModeling


if __name__ == "__main__":
    batch_size = 4


    from transformers import BertTokenizer, glue_convert_examples_to_features
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    # for gpu in gpus:
    #     tf.config.experimental.set_memory_growth(gpu, True)

    import tensorflow_datasets as tfds

    epochs = 5

    #create pretrain ds
#     pretrain_ds = load_pretrain_ds(tokenizer)
#     pretrain_ds =  pretrain_ds.shuffle(100)#.batch(batch_size)
#     for output in pretrain_ds:
#         print(list(output.keys()))
     #   print(output["document"])

    # create additional ds

    def train_understandable_only( understanding_dataset,only_dense = False, save_path = "model"):
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        model = Understandable_Embedder(batch_size, train_only_dense = only_dense)

        model.compile(optimizer=optimizer, loss=loss,metrics=["sparse_categorical_accuracy"])


        history = model.fit_understandable(understanding_dataset, epochs=epochs, steps_per_epoch=int(3600/batch_size)) #3600 datapoints
        path = "results/" + save_path
        os.makedirs(path,exist_ok=True)
        with open(path + "/history.txt", "wb") as fp:
            pickle.dump(history, fp)
        model.save_weights(path +"/model")

        return history

    def train_understandable_then_normal( train_dataset,understanding_dataset,only_dense = False, save_path = "model"):
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        model = Understandable_Embedder(batch_size, train_only_dense = only_dense)

        model.compile(optimizer=optimizer, loss=loss,metrics=["sparse_categorical_accuracy"])


        history = model.fit_understandable(understanding_dataset, epochs=epochs, steps_per_epoch=int(3600/batch_size)) #3600 datapoints
        history = model_normal.fit_classify(train_dataset, epochs=epochs, steps_per_epoch=int(3600/batch_size))
        path = "results/" + save_path
        os.makedirs(path,exist_ok=True)
        with open(path + "/history.txt", "wb") as fp:
            pickle.dump(history, fp)
        model.save_weights(path +"/model")

        return history

    def train_understandable(train_dataset, understanding_dataset,only_dense = False, save_path = "model", load_path = "noload", debias_freq = 1 ):
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        model = Understandable_Embedder(batch_size, train_only_dense = only_dense, debias_freq= debias_freq)
        if not load_path == "noload":
            model.load_weights(load_path)
        model.compile(optimizer=optimizer, loss=loss,metrics=["sparse_categorical_accuracy"])


        history = model.fit_classify_understandable(train_dataset,understanding_dataset, epochs=epochs, steps_per_epoch=int(3600/batch_size)) #3600 datapoints
        path = "results/" + save_path
        os.makedirs(path,exist_ok=True)
        with open(path + "/history.txt", "wb") as fp:
            pickle.dump(history, fp)
        model.save_weights(path +"/model")

        return history

    def train_normal(train_dataset,only_dense = False, save_path = "normal", load_path = "noload"):
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        model_normal = Understandable_Embedder(batch_size, train_only_dense = only_dense)#, debias_freq = debias_freq)
        if not load_path == "noload":
            model_normal.load_weights(load_path)
        model_normal.compile(optimizer=optimizer, loss=loss,metrics=["sparse_categorical_accuracy"])

      #  model.fit(train_dataset, epochs=2, steps_per_epoch=1840/batch_size)
        history = model_normal.fit_classify(train_dataset, epochs=epochs, steps_per_epoch=int(3600/batch_size))
        path = "results/" + save_path
        os.makedirs(path,exist_ok=True)
        with open(path +"/history.txt", "wb") as fp:
            pickle.dump(history, fp)
        model_normal.save_weights(path +"/model")

        return history

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states=True)

#     definition_pairs = [[[" good ", " positive "],[" bad ", " negative "]],
#                         [[" women ", " girl "],[" men ", " boy "]],
#                         [[" asian ", "  Asian "],[" caucasian "," Caucasian "],[" african "," African "],[" european "," European "],[" american ", " American "]],
#                         [[" Muslim ", " muslims "],[" Jew ", " jews "], [" Christian ", " christians "]]]
#
#     tokenized_definition_train_all = get_understanding_set(definition_pairs,tokenizer)


#     definition_pairs = [ [[" asian ", "  Asian "],[" caucasian "," Caucasian "],[" african "," African "],[" european "," European "],[" american ", " American "]]]
#
#     tokenized_definition_train_race = get_understanding_set(definition_pairs,tokenizer)
#
#     definition_pairs = [[[" women ", " girl "],[" men ", " boy "]]]
#
#     tokenized_definition_train_gender = get_understanding_set(definition_pairs,tokenizer)

    # definition_pairs = [[[" women ", " girl ", " female ", " she ", " actress ", " heroine ", " queen "," sister ", " mother ", " lady ", " her " ],[" men ", " boy ", " male ", " he ", " actor ", " hero ", " king ", " brother ", " father ", " gentleman ", " him "]]]

    # tokenized_definition_train_gender_large = get_understanding_set(definition_pairs,tokenizer)




    #iterate over glue tasks
    task_list = [  "sst2", "qnli"] # "sst2",["mrpc"]#,"cola"] #, "mnli"]"cola", "qnli", "sst2"
    for task in task_list:

        data = tfds.load('glue/'+task)
        if task == "sst2":
            task_name = "sst-2"
        else:
            task_name = task
        train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128,  task=task_name)
        train_dataset = train_dataset.shuffle(100).batch(batch_size).repeat(-1)
        for i in range(0,5):
            train_normal(train_dataset, save_path = (task+"fine/train"+str(i)))
          #  train_understandable(train_dataset,tokenized_definition_train_gender_large, save_path = (task+"_prefine_gender_"+str(i)), load_path = "results/pre_gender_/model")
           # train_understandable(train_dataset,tokenized_definition_train_gender_large, save_path = (task+"prefine_debfreq1/train"+str(i)), load_path = "results/pre_gender_5000/train1/model")
        # for a in range(4):
        #     for i in range(7):
        #       #  train_understandable(train_dataset,tokenized_definition_train_gender_large, save_path = (task+"_prefine_gender_"+str(i)), load_path = "results/pre_gender_/model")
        #         train_understandable(train_dataset,tokenized_definition_train_gender_large, save_path = (task+"Debias_freq/train"+str(a)+"_" +str(2**i)), debias_freq= 2**i)#, load_path = "results/pre_gender_5000/train1/model")
#       #  train_understandable_only(tokenized_definition_train_gender, save_path = (task+"only_understandable_gender"))
#       #  train_understandable(train_dataset,tokenized_definition_train_gender, save_path = ("gender_con_0"))
#         for i in range(5):
#            # train_normal(train_dataset, save_path = (task+ "_normal_"+str(i)))
#            train_understandable_only(tokenized_definition_train_gender, save_path = ("gender_"+str(i)))
#           # train_understandable(train_dataset,tokenized_definition_train_gender, save_path = (task+"_gender_"+str(i)))
#           # train_understandable_then_normal(train_dataset,tokenized_definition_train_gender, save_path = (task+"_staggered_gender_"+str(i)))
# #             train_understandable(train_dataset,tokenized_definition_train_race, save_path = ("race_con_"+str(i)))
#           # train_understandable(train_dataset,tokenized_definition_train_gender, save_path = (task+"_gender_con_"+str(i)))


ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model
ERROR: Could not find a version that satisfies the requirement Understandable_Embedder (from versions: none)
ERROR: No matching distribution found for Understandable_Embedder
ERROR: Invalid requirement: 'find_all_occurence_and_replace,load_pretrain_ds,get_understanding_set'


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Shuffling /root/tensorflow_datasets/glue/sst2/2.0.0.incomplete0WBEGJ/glue-test.tfrecord*...:   0%|          | …

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/sst2/2.0.0. Subsequent calls will reuse this data.


All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


At epoch 1 of 5
Average Training loss at step 20 / 900 : 0.7895196782425046
Average Training loss at step 40 / 900 : 0.6965225487947464
Average Training loss at step 60 / 900 : 0.6405819252133369
Average Training loss at step 80 / 900 : 0.4729052439332008
Average Training loss at step 100 / 900 : 0.587383546680212
Average Training loss at step 120 / 900 : 0.5913736067712307
Average Training loss at step 140 / 900 : 0.47337839007377625
Average Training loss at step 160 / 900 : 0.46665443293750286
Average Training loss at step 180 / 900 : 0.39601098373532295
Average Training loss at step 200 / 900 : 0.3344112548977137
Average Training loss at step 220 / 900 : 0.3276843633502722
Average Training loss at step 240 / 900 : 0.36251271767541765
Average Training loss at step 260 / 900 : 0.2187945380806923
Average Training loss at step 280 / 900 : 0.4492562051862478
Average Training loss at step 300 / 900 : 0.27647268017753956
Average Training loss at step 320 / 900 : 0.41375787910073997
Average

KeyboardInterrupt: ignored

In [21]:
import matplotlib.pyplot as plt
!pip install model
!pip install Understandable_Embedder
!pip install utils
!pip install DataLoader, SentGenerator
!pip install bias_eval import BiasMetrics, WEATMetrics, WEAT, DistanceTests, ImprovedBiasMetrics
import copy
import numpy as np
from numpy import linalg as LA
import math
!pip install data import get_understanding_set
from transformers import BertTokenizer, glue_convert_examples_to_features
from nltk.corpus.reader import lin

def plot_history(path):
    with open(path + "/understandable/understandable_history.txt", "rb") as fp:   # Unpickling
        understandable_history = pickle.load(fp)
    with open(path +"/understandable/normal_history.txt", "rb") as fp:   # Unpickling
        normal_history = pickle.load(fp)
  #  plt.plot(understandable_history["loss_compare"], label = "loss_compare")
    plt.plot(understandable_history["loss"], label = "loss")
    plt.plot(normal_history['loss'], label = "loss_original")
    plt.legend()
   # plt.plot(normal_history['sparse_categorical_accuracy'], label = "accuracy")
    plt.savefig("History_race_gender.eps")
    plt.show()


def plot_average_history(path, intervall):

#     history_list_gender=[]
#     for i in range(intervall):
#         with open(path +"cola_gender_large_" + str(i) + "/history.txt", "rb") as fp:   # Unpickling
#             understandable_history = pickle.load(fp)
#             history_list_gender.append(understandable_history)
#     generate_error_bound_plot(history_list_gender)

    history_list_race =[]
    for i in range(intervall):
        with open(path +"Genderlarge/sst2_gender_large_" + str(i) + "/history.txt", "rb") as fp:   # Unpickling
            understandable_history = pickle.load(fp)
            history_list_race.append(understandable_history)
    generate_error_bound_plot(history_list_race, color=(0.1,0.6,0.1), label = "debiased")
#
    history_list_normal=[]
    for i in range(intervall):
        with open(path +"without/sst2_normal_" + str(i) + "/history.txt", "rb") as fp:   # Unpickling
            understandable_history = pickle.load(fp)
            history_list_normal.append(understandable_history)
    generate_error_bound_plot(history_list_normal, color=(0.6,0.1,0.1), label = "original")

    plt.savefig("average_history_sst2_both.png")

def generate_error_bound_plot(history_list, color=(0.1,0.1,0.6), label = ""):
    history_matrix=np.zeros((len(history_list),len(history_list[0]["loss"])))
    for i,history in enumerate(history_list):
        history_matrix[i,:]=history["loss"]
    mean_history = np.mean(history_matrix,axis=0)
    var_history = np.var(history_matrix,axis=0)
    confidence_intervall = 1.96*np.sqrt(var_history)/np.sqrt(len(history_list))
    confidence_intervall_upper = mean_history + confidence_intervall
    confidence_intervall_lower =   mean_history - confidence_intervall

    boundcolor = (color[0]*1.5,color[1]*1.5,color[2]*1.5)
    line, = plt.plot(mean_history,c = color)
    if not label == "":
        line.set_label(label)
        plt.legend()
    plt.plot(confidence_intervall_upper,c = boundcolor)
    plt.plot(confidence_intervall_lower,c = boundcolor)

def load_model(modelpath):
    model = Understandable_Embedder()
    model.load_weights(modelpath)
    return model

def evaluate_model_accuracy(modelpath, eval_ds, dataset_length):
    model = Understandable_Embedder()
    if not modelpath == "":
        model.load_weights(modelpath)

    eval_ds = eval_ds.batch(batch_size)
    return model.evaluate(eval_ds, batch_size, dataset_length)

def evaluate_model_accuracy_project(modelpath, eval_ds, dataset_length, P):
    model = Understandable_Embedder()
    if not modelpath == "":
        model.load_weights(modelpath)

    eval_ds = eval_ds.batch(batch_size)
    return model.eval_simple_project(eval_ds, batch_size, dataset_length,P)

def evaluate_average_model_accuracy(path,eval_ds, dataset_length, intervall):
    average = 0.0
    for i in range(intervall):
        average = average + evaluate_model_accuracy(path + str(i)+"/model", eval_ds, dataset_length)
    average = average / intervall
    return average

def evaluate_model_bias(modelpath, savepath, delete_dim_bool=False, pca_deb= False):


    # load the tf news dataset
    data_loader = DataLoader()#news_sent = data_loader.load_news_dataset()


    # professions and company names as target words
    prof = data_loader.read_target_words_from_json('exp_data/target_words/professions.json', index=0)
    comp = data_loader.read_target_words_from_json('exp_data/target_words/companies.json')
    sent_test = data_loader.read_target_words_from_json('exp_data/sentiment/test_words.json')

    # bias attributes for gender, sentiment and religion
    gender_attr = data_loader.read_bias_attributes_from_json('exp_data/gender/definitional_pairs.json')
    religion_attr = data_loader.read_bias_attributes_from_json('exp_data/religion/definitional_pairs.json')
    sentiment_attr = data_loader.read_bias_attributes_from_json('exp_data/sentiment/definitional_pairs.json')
    race_attr = data_loader.read_bias_attributes_from_json('exp_data/race/definitional_pairs.json')

    weats = {}
    for test in ['weat3', 'weat3b', 'weat4', 'weat5', 'weat5b', 'weat6', 'weat6b', 'weat7', 'weat7b', 'weat8', 'weat8b']:
        file = 'exp_data/seat/'+test+'.jsonl'
        print("read: "+file)
        weats.update({test: data_loader.read_dict_from_json(file)})
    weats.update({'angry_black_woman': data_loader.read_dict_from_json('exp_data/seat/angry_black_woman_stereotype.jsonl')})
    weats.update({'angry_black_woman2': data_loader.read_dict_from_json('exp_data/seat/angry_black_woman_stereotype_b.jsonl')})
    weats.update({'double_bind_competent': data_loader.read_dict_from_json('exp_data/seat/heilman_double_bind_competent_one_word.jsonl')})
    weats.update({'double_bind_likable': data_loader.read_dict_from_json('exp_data/seat/heilman_double_bind_likable_one_word.jsonl')})

    # read sentence templates and replacement rules
    template_dict = data_loader.read_dict_from_json('exp_data/templates.json')
    rules_dict = data_loader.read_dict_from_json('exp_data/template_rules.json')

    sent_generator = SentGenerator()

    print("test sentences (target words):")
    test_sent_sentiment = sent_generator.create_sent_from_template_dict(sent_test, template_dict, rules_dict, keys=['adjectives', 'things_singular', 'things_no_pronoun'])
    test_sent_prof = sent_generator.insert_to_templates(prof, template_dict['people_singular'], template_dict['token'])
    test_sent_comp = sent_generator.insert_to_templates(comp, template_dict['things_no_pronoun'], template_dict['token'])

    print("bias attribute sentences:")
    btest_sent_gender = sent_generator.create_attr_sent_from_template_dict(list(zip(*gender_attr)), template_dict, rules_dict)
    #print(btest_sent_gender)
    btest_sent_religion = sent_generator.create_attr_sent_from_template_dict(list(zip(*religion_attr)), template_dict, rules_dict)
    btest_sent_sentiment = sent_generator.create_attr_sent_from_template_dict(list(zip(*sentiment_attr)), template_dict, rules_dict)
    btest_sent_race = sent_generator.create_attr_sent_from_template_dict(list(zip(*race_attr)), template_dict, rules_dict)


    # create sentences for SEATs
    def create_seat_sent(weat_dict, template_dict, rules_dict):
        seat_dict = copy.deepcopy(weat_dict)
        for test in weat_dict.keys():
            for k in weat_dict[test].keys():
                category = weat_dict[test][k]['category']
                if 'Names' in category or category in ['Male', 'Female']:
                    seat_dict[test][k]['examples'] = sent_generator.insert_to_templates(weat_dict[test][k]['examples'], template_dict['names'], template_dict['token'])
                elif category in ['EuropeanAmericanTerms', 'AfricanAmericanTerms']:
                    seat_dict[test][k]['examples'] = sent_generator.insert_to_templates(weat_dict[test][k]['examples'], template_dict['people_description'], template_dict['token'])
                else:
                    seat_dict[test][k]['examples'] = sent_generator.create_sent_from_template_dict(weat_dict[test][k]['examples'], template_dict, rules_dict)
        return seat_dict

    seats = create_seat_sent(weats, template_dict, rules_dict)

    def embed_sent(embed_function):
        print("embed gender...")
        emb_gender = [embed_function(tup) for tup in btest_sent_gender]
        print("embed religion...")
        emb_religion = [embed_function(tup) for tup in btest_sent_religion]
        print("embed race...")
        emb_race = [embed_function(tup) for tup in btest_sent_race]
        print("embed sentiment...")
        emb_sentiment_tup = [embed_function(tup) for tup in btest_sent_sentiment]

        for k, v in seats.items():
            print("embed "+k+"...")
            for v2 in v.values():
                v2['embeddings'] = embed_function(v2['examples'])

        return {'gender': emb_gender, 'religion': emb_religion, 'race': emb_race, 'sentiment': emb_sentiment_tup}

    def cossim(x, y):
        return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

    def mac(T, As):
        return np.mean([1-cossim(t,a) for t in T for A in As for a in A])

    def normalize(vectors: np.ndarray):
        norms = np.apply_along_axis(LA.norm, 1, vectors)
        vectors = vectors / norms[:, np.newaxis]
        return np.asarray(vectors)

    def delete_dim(x,dims=[0]):
        for dim in dims:
           # print(x.shape)
            x = np.concatenate((x[:,:dim],x[:,dim+1:]), axis = 1)
           # print(x.shape)
        return x

    def compare_metrics(seats,log_file): # embeddings
        f = open(log_file,"w")
        print("opened file", log_file)
        f.write("test;eff_weat;pval_weat;eff_own;own_bias_mean;own_std;mac;cluster\n")
        result_dict = {}
        for test in seats.keys():
            if not ('embeddings' in seats[test]['attr1'].keys() and 'embeddings' in seats[test]['targ1'].keys() and 'embeddings' in seats[test]['attr2'].keys() and 'embeddings' in seats[test]['targ2'].keys()):
                print("skip "+test)
                continue


            # normalize embeddings
            seats[test]['attr1']['embeddings'] = normalize(seats[test]['attr1']['embeddings'])
            seats[test]['attr2']['embeddings'] = normalize(seats[test]['attr2']['embeddings'])
            seats[test]['targ1']['embeddings'] = normalize(seats[test]['targ1']['embeddings'])
            seats[test]['targ2']['embeddings'] = normalize(seats[test]['targ2']['embeddings'])

            print(test)

            pval, esize = weat.run_tests(seats[test])

            esize2, bias_mean, bias_std = weat.run_tests_own(seats[test])

            mac_score = mac(seats[test]['targ1']['embeddings']+seats[test]['targ2']['embeddings'], [seats[test]['attr1']['embeddings'],seats[test]['attr2']['embeddings']])

            X_bef = np.vstack([seats[test]['targ1']['embeddings'],seats[test]['targ2']['embeddings']])
            y = np.asarray([0]*len(seats[test]['targ1']['embeddings'])+[1]*len(seats[test]['targ2']['embeddings']))
          #  cluster_score = distance_tests.cluster_test(X_bef, y, num=2)

            f.write(test+"&"+str(esize)+"&"+str(pval)+";&"+str(esize2)+"&"+str(bias_mean)+"&"+str(bias_std)+"&"+str(mac_score)+"\\"+"\n")

            test_list = []
            test_list.append(esize)
            test_list.append(esize2)
            test_list.append(mac_score)
            test_list.append(pval)


            result_dict[test] = test_list
            # get single word biases
            #W = np.vstack([weats[test]['targ1']['embeddings'],weats[test]['targ2']['embeddings']])
            #word_biases = [weat.own_metric.bias_w(w, A) for w in W]

        # TODO run other tests

        f.close()

        return result_dict

    def delete_dim_seats(seats,dims=[0]):
        for test in seats.keys():
            if not ('embeddings' in seats[test]['attr1'].keys() and 'embeddings' in seats[test]['targ1'].keys() and 'embeddings' in seats[test]['attr2'].keys() and 'embeddings' in seats[test]['targ2'].keys()):
                print("skip "+test)
                continue

            seats[test]['attr1']['embeddings'] = delete_dim(seats[test]['attr1']['embeddings'],dims)
            seats[test]['attr2']['embeddings'] = delete_dim(seats[test]['attr2']['embeddings'],dims)
            seats[test]['targ1']['embeddings'] = delete_dim(seats[test]['targ1']['embeddings'],dims)
            seats[test]['targ2']['embeddings'] = delete_dim(seats[test]['targ2']['embeddings'],dims)

        return seats


    weat_metrics = WEATMetrics()
    own_metrics = ImprovedBiasMetrics()
    weat = WEAT(weat_metrics, own_metrics)

    model = Understandable_Embedder()
    if not modelpath == "original":
        model.load_weights(modelpath)
    #model.call_headless(inputs)

  #  print(model.predict_simple(["hallo du da", "ich bin hier"]))
    def emb_func(input):
      x =   model.predict_simple(input)
      return x

    if pca_deb:
        from SentDebias import Sent_Debias
        Sent_Deb = Sent_Debias()
        Sent_Deb.load()
        def emb_func(input):
            x =   model.predict_simple(input)
            x = Sent_Deb.predict(x)
            return x
    emb = embed_sent(emb_func)
  #  print(emb)

   # print(delete_dim)
    if delete_dim_bool:
        seats = delete_dim_seats(seats)

    result_dict = compare_metrics(seats,savepath)


    # 3-5b european american vs african american pleasant vs unpleasant
    # 6-8b  gender
    # angry black woman racial
    # double bind gender
    return result_dict

def word_correlation(word_pair, modelpath, dim = 0):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states=True)
    dataset = get_understanding_set(word_pair,tokenizer, max_examples = 1000)

    model = Understandable_Embedder()
    model.load_weights(modelpath)

    batch_size = 4
    steps = round(len(dataset[0][0]["input_ids"]) / batch_size)

    for i in range(0,steps):
        feed_dict_1 = {}
        feed_dict_1["input_ids"] = dataset[0][0]["input_ids"][i*batch_size:(i+1)*batch_size]
        feed_dict_1["token_type_ids"] = dataset[0][0]["token_type_ids"][i*batch_size:(i+1)*batch_size]
        feed_dict_1["attention_mask"] = dataset[0][0]["attention_mask"][i*batch_size:(i+1)*batch_size]

        feed_dict_2 = {}
        feed_dict_2["input_ids"] = dataset[0][1]["input_ids"][i*batch_size:(i+1)*batch_size]
        feed_dict_2["token_type_ids"] = dataset[0][1]["token_type_ids"][i*batch_size:(i+1)*batch_size]
        feed_dict_2["attention_mask"] = dataset[0][1]["attention_mask"][i*batch_size:(i+1)*batch_size]

        if i == 0 :
            original_emb = model.call_headless(feed_dict_1)
            changed_emb = model.call_headless(feed_dict_2)
        original_emb = np.concatenate((original_emb,model.call_headless(feed_dict_1)))
        changed_emb = np.concatenate((changed_emb,model.call_headless(feed_dict_2)))


#     print(original_emb[:,dim])
#     print(changed_emb[:,dim])
#     print(original_emb[:,dim]- changed_emb[:,dim])
    change = np.mean(np.abs(original_emb[:,dim]- changed_emb[:,dim]))
    change_other = np.sum(np.mean(np.abs(original_emb[:,dim:]- changed_emb[:,dim:]), axis = 0), axis = 0)

    print("change:",change)
    print("change in other dimension:",change_other)
    print("percent in right dim:", change/(change+change_other))

def evaluate_model_set(model_path, intervall_range, save_path = "results/finetuned.txt", delete_dim = False):

    if intervall_range == 0:
        result_dict = {}
        dict = evaluate_model_bias(model_path +"/model", "results/finetuned.txt", delete_dim_bool= delete_dim)
        for test in dict:
            if not test in result_dict:
                result_dict[test] =[]
            result_dict[test].append(dict[test])
    else:
        result_dict = {}
        for i in range(intervall_range):
            dict = evaluate_model_bias(model_path+str(i) +"/model", "results/finetuned.txt", delete_dim_bool= delete_dim)
            for test in dict:
                if not test in result_dict:
                    result_dict[test] =[]
                result_dict[test].append(dict[test])

    log_file = save_path
    f = open(log_file,"w")
    print("opened file", log_file)
    f.write("test & weat & own & mac\\\n")

    average_dict = {}
    for test in result_dict:
        result_dict[test] = np.asarray(result_dict[test])
        variance = np.var(result_dict[test], axis = 0 )
        average_dict[test + "intervall"] = 1.96*np.sqrt(variance)/np.sqrt(result_dict[test].shape[0])
        average_dict[test] = np.mean(np.abs(result_dict[test]), axis = 0)
    #    print(test, average_dict[test], average_dict[test + "intervall"] )
        f.write(test+" & "+str(round(average_dict[test][0], 2))+ " +/- " + str(round(average_dict[test + "intervall"][0], 2)) +" & "
                +str(round(average_dict[test][1],2))+ " +/- " + str(round(average_dict[test + "intervall"][1], 2)) +" & "
                +str(round(average_dict[test][2],2))+ " +/- " + str(round(average_dict[test + "intervall"][2], 2)) +" & "
                +str(round(average_dict[test][3],2))+ " +/- " + str(round(average_dict[test + "intervall"][3], 2)) +"\\"+"\n")





    race_test = ["weat3", "weat3b", "weat4", "weat5", "weat5b", "angry_black_woman", "angry_black_woman2"]
    gender_test = ["weat6", "weat6b", "weat7", "weat7b", "weat8b", "weat8"]#, "double_bind_competent", "double_bind_likable"]

    race_list =[]
    for test in race_test:
        race_list.append(average_dict[test])
    race_score = np.mean(race_list,axis = 0)

    gender_list =[]
    for test in gender_test:
        gender_list.append(average_dict[test])
    gender_score = np.mean(gender_list,axis = 0)

    f.write("gender_score: "+ str(gender_score) + "\n")
    f.write("race_score: "+ str(race_score) + "\n")


    f.close()

def generate_sentences(words):
    templates =[
        "I am a ",
        "I work as a ",
        "My occupation is ",
        "My dream job is ",
        "works as a ",
        "wants to become a ",
        "Overall i am quite happy with being a ",
        "I am depressed working as a ",
        "In conclusion i am neutral towards my job as a ",
        "I am working as a ",
        "This is my job: ",
        "I am conflicted about being a "
        ]
    sentences =[]
    for word in words:
        new_sentences = []
        for template in templates:
            new_sentences.append(template + word)
        sentences.append(new_sentences)
    return sentences

def plot_tsne(model):
    import json
#     with open("professions.json") as f:
#         list = json.load(f)
#
#     x = []
#     y = []
#     for element in list:
#         x.append(element[0])
#         y.append(element[2])
    # x = ["women", "man", "boy", "girl"]
    # y = [-1,1,1,-1]
    # y = np.asarray(y)
    # y = y*0.5+0.5
    # X = generate_sentences(x)
    # new_X = []
    # for batch in X:
    #     new_X.append(model.predict_simple(batch))
    # X = np.asarray(new_X)
    # X = np.mean(X, axis = 1)
#     y = np.asarray(y)
#     y = y*0.5+0.5 #scale to 0-1
    #X = generate_sentences(x)
    x = [" women ", " girl ", " female ", " she ", " actress ", " heroine ", " queen "," sister ", " mother ", " lady ", " her " ," men ", " boy ", " male ", " he ", " actor ", " hero ", " king ", " brother ", " father ", " gentleman ", " him "]
    y = [0]*11
    y2 = [1]*11
    y = y + y2
    X = model.predict_simple(x)
    print(X)
#
#     new_X = []
#     for batch in X:
#         new_X.append(model.predict_simple(batch))
#     X = np.asarray(new_X)
#     X = np.mean(X, axis = 1)

    from sklearn.manifold import TSNE
    X_embedded = TSNE(n_components=2).fit_transform(X.astype(np.float64))
    plt.scatter(X_embedded[:,0], X_embedded[:,1], c = y, cmap = plt.get_cmap("winter"))
    plt.show()
    return

def gender_bias_test(model,pca_deb=False, null = False):
    import json
    with open("professions.json") as f:
        list = json.load(f)

    x = []
    y = []
    for element in list:
        x.append(element[0])
        y.append(element[2])



    X = generate_sentences(x)

    new_y = []
    new_x =[]
    for i,batch in enumerate(X):
        for i in range(len(batch)):
            new_y.append(y[i])
        for sentence in batch:
            new_x.append(sentence)

    y = new_y
    y = np.asarray(y)
    y = y*0.5+0.5
    X = new_x

    print("base classifier: mse loss: " , np.var(y))

#     X_test = generate_sentences(X_test)
#     new_y = []
#     new_x = []
#     for i,batch in enumerate(X_test):
#         for i in range(len(batch)):
#             new_y.append(y_test[i])
#         for sentence in batch:
#             new_x.append(sentence)
#
#     y_test = new_y
#     y_test = np.asarray(y_test)
#     y_test = y_test*0.5+0.5
#     X_test = new_x

    X = model.predict_simple(X)

    if null:
   #     y_class = np.asarray([np.round(label) for label in y])
        P = nullspace(model)
        X = np.transpose(np.matmul(P,np.transpose(X)))
 #   X_test = model.predict_simple(X_test)
  #  print(len(y))

    if pca_deb:
        from SentDebias import Sent_Debias
        Sent_Deb = Sent_Debias()
        Sent_Deb.load()
        X = Sent_Deb.predict(X)
     #   X_test = Sent_Deb.predict(X_test)

    from sklearn.model_selection import train_test_split
    X, X_test,y, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

    from tensorflow import keras
    simple_model = keras.Sequential(
    [
        keras.Input(shape=X[0].shape),
        keras.layers.Dense(1, activation = "sigmoid")
    ])
    simple_model.compile(optimizer =tf.keras.optimizers.SGD() ,loss = tf.keras.losses.MeanSquaredError())
    simple_model.fit(X,y, batch_size = 8, epochs = 50, validation_data = (X_test,y_test), verbose = 0)#, callbacks = keras.callbacks.EarlyStopping())
    simple_result = simple_model.evaluate(X_test,y_test)
    print("linear model:",simple_result)



    medium_model = keras.Sequential(
    [
        keras.Input(shape=X[0].shape),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(1, activation = "sigmoid")
    ])
    medium_model.compile(optimizer =tf.keras.optimizers.SGD() ,loss = tf.keras.losses.MeanSquaredError())
    medium_model.fit(X,y, batch_size = 8, epochs = 50, validation_data = (X_test,y_test), verbose = 0)#, callbacks = keras.callbacks.EarlyStopping())
    medium_result = medium_model.evaluate(X_test,y_test)
    print("not as simple model:",medium_result)

    return simple_result, medium_result


def gender_bias_test_classification(model,pca_deb=False):
    import json
    with open("professions.json") as f:
        list = json.load(f)

    x = []
    y = []
    for element in list:
        x.append(element[0])
        y.append(element[2])



    X = generate_sentences(x)

    new_y = []
    new_x =[]
    for i,batch in enumerate(X):
        for i in range(len(batch)):
            new_y.append(y[i])
        for sentence in batch:
            new_x.append(sentence)

    y = new_y
    y = np.asarray(y)
    y = y*0.5+0.5
    y = np.asarray([np.round(label) for label in y])

    X = new_x

    print("base classifier: mse loss: " , np.var(y))

#     X_test = generate_sentences(X_test)
#     new_y = []
#     new_x = []
#     for i,batch in enumerate(X_test):
#         for i in range(len(batch)):
#             new_y.append(y_test[i])
#         for sentence in batch:
#             new_x.append(sentence)
#
#     y_test = new_y
#     y_test = np.asarray(y_test)
#     y_test = y_test*0.5+0.5
#     X_test = new_x

    X = model.predict_simple(X)
 #   X_test = model.predict_simple(X_test)
  #  print(len(y))

    if pca_deb:
        from SentDebias import Sent_Debias
        Sent_Deb = Sent_Debias()
        bias_words = ["girl", "boy"]
        Sent_Deb.fit(model,bias_words ,k= 3, max_words = 1000)
        X = Sent_Deb.predict(X)
     #   X_test = Sent_Deb.predict(X_test)

    from sklearn.model_selection import train_test_split
    X, X_test,y, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

    print(X_test.shape)
    print(y_test.shape)

    from tensorflow import keras
    simple_model = keras.Sequential(
    [
        keras.Input(shape=X[0].shape),
        keras.layers.Dense(1, activation = "sigmoid")
    ])
    simple_model.compile(optimizer =tf.keras.optimizers.SGD() ,loss = tf.keras.losses.BinaryCrossentropy())
    simple_model.fit(X,y, batch_size = 8, epochs = 50, validation_data = (X_test,y_test))#, callbacks = keras.callbacks.EarlyStopping())
    print("linear model:",simple_model.evaluate(X_test,y_test))


    # --- Deep View Parameters ----#
    from deepview import DeepView
    pred_wrapper = DeepView.create_simple_wrapper(simple_model.predict)

    classes = np.arange(2)
    batch_size = 32
    max_samples = 500
    data_shape = (768,)
    resolution = 100
    N = 10
    lam = 0.64
    cmap = 'tab10'
    # to make shure deepview.show is blocking,
    # disable interactive mode
    interactive = False
    title = 'linear model - occupations'

    deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, data_shape,
        N, lam, resolution, cmap, interactive, title)

    deepview.add_samples(X_test[:200], y_test[:200])
    deepview.show()



    medium_model = keras.Sequential(
    [
        keras.Input(shape=X[0].shape),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(20, activation="relu"),
        keras.layers.Dense(1, activation = "sigmoid")
    ])
    medium_model.compile(optimizer =tf.keras.optimizers.SGD() ,loss = tf.keras.losses.BinaryCrossentropy())
    medium_model.fit(X,y, batch_size = 8, epochs = 50, validation_data = (X_test,y_test))#, callbacks = keras.callbacks.EarlyStopping())
    print("not as simple model:",medium_model.evaluate(X_test,y_test))

    pred_wrapper = DeepView.create_simple_wrapper(medium_model.predict)

    classes = np.arange(2)
    batch_size = 32
    max_samples = 500
    data_shape = (768,)
    resolution = 100
    N = 10
    lam = 0.64
    cmap = 'tab10'
    # to make shure deepview.show is blocking,
    # disable interactive mode
    interactive = False
    title = 'nonlinear model - occupations'

    deepview = DeepView(pred_wrapper, classes, max_samples, batch_size, data_shape,
        N, lam, resolution, cmap, interactive, title)

    deepview.add_samples(X_test[:200], y_test[:200])
    deepview.show()

from nullspaceproject import get_debiasing_projection
from sklearn.linear_model import SGDClassifier, Perceptron, LogisticRegression
def nullspace(model):
    import json
    with open("professions.json") as f:
        list = json.load(f)

    x = []
    y = []
    for element in list:
        x.append(element[0])
        y.append(element[2])



    X = generate_sentences(x)

    new_y = []
    new_x =[]
    for i,batch in enumerate(X):
        for i in range(len(batch)):
            new_y.append(y[i])
        for sentence in batch:
            new_x.append(sentence)

    y = new_y
    y = np.asarray(y)
    y = y*0.5+0.5
    X = new_x


#     X_test = generate_sentences(X_test)
#     new_y = []
#     new_x = []
#     for i,batch in enumerate(X_test):
#         for i in range(len(batch)):
#             new_y.append(y_test[i])
#         for sentence in batch:
#             new_x.append(sentence)
#
#     y_test = new_y
#     y_test = np.asarray(y_test)
#     y_test = y_test*0.5+0.5
#     X_test = new_x

    X = model.predict_simple(X)

    y = np.asarray([np.round(label) for label in y])

    num_classifiers = 20
    classifier_class = SGDClassifier #Perceptron
    input_dim = 768
    is_autoregressive = True
    min_accuracy = 0.0

    P, rowspace_projections, Ws = get_debiasing_projection(classifier_class, {}, num_classifiers, input_dim, is_autoregressive, min_accuracy, X, y, X, y, by_class = False)

    return P

if __name__ == "__main__":

    #plot_average_history("results/", 5)



    #plot_history("race_modelmrpc")
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    # for gpu in gpus:
    #     tf.config.experimental.set_memory_growth(gpu, True)
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    import tensorflow_datasets as tfds
    from transformers import BertTokenizer, glue_convert_examples_to_features
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states=True)
    batch_size = 4

    plot_tsne(model = Understandable_Embedder())
   # plot_average_history("results/", 2)

#     model = Understandable_Embedder()
#     tokenized_inputs = tokenizer(["hallo du da", "ich bin hier"], max_length=128, padding=True, truncation=True, return_tensors='tf')
#     print(model.call_pre_training(tokenized_inputs))
#
#    evaluate_model_bias("results/pre_gender_5000/train1/model", savepath= "bertpre.txt" )
#     evaluate_model_set("results/Genderlarge/qnli_gender_large_", 5, save_path = "qnlifine.txt" )
#     evaluate_model_set("results/Genderlarge/sst2_gender_large_", 5, save_path = "sst2fine.txt" )
# #     evaluate_model_set("results/Genderlarge/cola_gender_large_", 5, save_path = "colafine.txt" )
#     model = Understandable_Embedder()
# #     # gender_bias_test(model = model, pca_deb= True)
#     plot_tsne(model)
#     model = load_model("results/qnli_prefine_gender_0/model")
#     plot_tsne(model)




    task = "cola"
    if task == "sst2":
        task_name = "sst-2"
    else:
        task_name = task
    data = tfds.load('glue/'+task)
    dataset_length = data['validation'].cardinality().numpy()
    dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128,  task=task_name)
  #  acc = evaluate_model_accuracy("results/colafine/train3/model", dataset, dataset_length)
   # P = np.identity(768)

    accs =[]
    lin_biases = []
    nonlin_biases = []
    for i in range(1):
        model_load_path = "results/"+task+"fine/train"+str(i)+ "/model"
        model = load_model(model_load_path)
       # model = Understandable_Embedder()
        P = nullspace(model)
        acc = evaluate_model_accuracy_project(model_load_path, dataset, dataset_length, P)
        accs.append(acc)
        lin_bias, nonlin_bias = gender_bias_test(model = model, null= True)
        lin_biases.append(lin_bias)
        print("lin_bias", lin_bias)
        nonlin_biases.append(nonlin_bias)
        print("nonlin_bias", nonlin_bias)
        print("acc", acc)

    def printmeanvar(list):
        mean = np.mean(list)
        std = np.std(list)
        print("mean", mean)
        print("std", std)
    print("accs")
    printmeanvar(accs)
    print("lin_biases")
    printmeanvar(lin_biases)
    print("nonlin_biases")
    printmeanvar(nonlin_biases)
    print("extra")
#    P = nullspace(model)
  #  P = np.identity(768)
   # acc = model.eval_simple_project(dataset, P, dataset_length)
  #  print(acc)
  #
#     model = load_model("results/Genderlarge/cola_gender_large_0/model")
#     gender_bias_test_classification(model = model)
#     model = load_model("results/Genderlarge/cola_gender_large_1/model")
#     gender_bias_test_classification(model = model)

#     model = load_model("results/cola_prefine_gender_0/model")
#     gender_bias_test_classification(model = model, pca_deb= True)
#     model = load_model("results/without/cola_normal_1/model")
#     gender_bias_test_classification(model = model)
#     model = load_model("results/Genderlarge/sst2_gender_large_0/model")
#     gender_bias_test_classification(model = model)
#     model = load_model("results/Genderlarge/sst2_gender_large_1/model")
#     gender_bias_test_classification(model = model)
   # print("not pre trained:")
#     model = Understandable_Embedder()
#     gender_bias_test(model = model, pca_deb = True)

#     model = load_model("results/pre_gender_50000/model")
#     lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#
#

#     f =open('results_debiasfreq_search.txt', 'w')
#
#     num = 4
#     overalllin = []
#     overallnonlin = []
#     for a in range(7):
#         linavg = []
#         nonlinavg = []
#         for i in range(5):
#             model = load_model("results/qnliDebias_freq/train"+str(i) +"_"+ str(2**a)+ "/model")
#             lin,nonlin = gender_bias_test(model = model, pca_deb = False)
#             linavg.append(lin)
#             nonlinavg.append(nonlin)
#
#         overalllin.append(linavg)
#         overallnonlin.append(nonlinavg)
#         print("ratio ", 2**a, file = f)
#         print("linavg mean", np.mean(linavg), file = f)
#         print("nonlinavg mean", np.mean(nonlinavg), file = f)
#         print("linavg var", np.var(linavg), file = f)
#         print("nonlinavg var", np.var(nonlinavg), file = f)
#
#     f.close()
#     with open('linandnonlin.npy', 'wb') as f:
#         np.save(f, overalllin)
#         np.save(f, overallnonlin)
#     linavg = 0
#     nonlinavg = 0
#     for i in range(num):
#         model = load_model("results/sst2prefine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/num, file = f)
#     print("nonlinavg", nonlinavg/num, file = f)
#
#     linavg = 0
#     nonlinavg = 0
#     for i in range(num):
#         model = load_model("results/qnliprefine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/num, file = f)
#     print("nonlinavg", nonlinavg/num, file = f)
#
#     linavg = 0
#     nonlinavg = 0
#     for i in range(num):
#         model = load_model("results/colapre_lessfine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/num, file = f)
#     print("nonlinavg", nonlinavg/num, file = f)
#
#     linavg = 0
#     nonlinavg = 0
#     for i in range(num):
#         model = load_model("results/sst2pre_lessfine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/num, file = f)
#     print("nonlinavg", nonlinavg/num, file = f)
#
#     linavg = 0
#     nonlinavg = 0
#     for i in range(num):
#         model = load_model("results/qnlipre_lessfine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/num, file = f)
#     print("nonlinavg", nonlinavg/num, file = f)



#     for i in range(5):
#         model = load_model("results/Genderlarge/sst2_gender_large_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/5.0)
#     print("nonlinavg", nonlinavg/5.0)
#
#
#     for i in range(5):
#         model = load_model("results/Genderlarge/qnli_gender_large_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         linavg += lin
#         nonlinavg += nonlin
#     print("linavg", linavg/5.0)
#     print("nonlinavg", nonlinavg/5.0)


#     task = "cola"
#     data = tfds.load('glue/'+task)
#     dataset_length = data['test'].cardinality().numpy()
#     dataset = glue_convert_examples_to_features(data['test'], tokenizer, max_length=128,  task=task)
#
#     gender_acc = evaluate_average_model_accuracy("results/cola_prefine_gender_", dataset, dataset_length, 2)
#     print("gender_acc cola:", gender_acc)
#
#     gender_acc = evaluate_average_model_accuracy("results/cola_prefine_without_gender_", dataset, dataset_length, 2)
#     print("gender_acc cola:", gender_acc)


#     task = "sst2"
#     data = tfds.load('glue/'+task)
#     task = "sst-2"
#     dataset_length = data['validation'].cardinality().numpy()
#     dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128,  task=task)
#     gender_acc = evaluate_average_model_accuracy("results/sst2_prefine_without_gender_", dataset, dataset_length, 2)
#     print("gender_acc sst2:", gender_acc)
#     gender_acc = evaluate_average_model_accuracy("results/sst2_prefine_gender_", dataset, dataset_length, 2)
#     print("gender_acc sst2:", gender_acc)
#

#     f =open('results.txt', 'w ')

#     dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128,  task=task)
#     gender_acc = evaluate_average_model_accuracy("results/qnli_prefine_gender_", dataset, dataset_length, 5)
#     print("gender_acc qnli:", gender_acc, file = f)
#     f =open('resultscolabias.txt', 'w')
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/colaBaselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/colaBaselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert Sent",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/colaprefine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert prefine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/Genderlarge/cola_gender_large_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert fine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#
#     f.close()
#
#     f =open('resultssst2bias.txt', 'w')
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/sst2Baselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/sst2Baselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert Sent",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/sst2prefine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert prefine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/Genderlarge/sst2_gender_large_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert fine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#
#     f.close()
#
#     f =open('resultsqnlibias.txt', 'w')
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/qnliBaselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/qnliBaselines/_baseline_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model, pca_deb = True)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert Sent",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/qnliprefine_debfreq1/train" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert prefine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#     avglin = []
#     avgnonlin = []
#     for i in range(5):
#         model = load_model("results/Genderlarge/qnli_gender_large_" + str(i)+ "/model")
#         lin,nonlin = gender_bias_test(model = model)
#         avglin.append(lin)
#         avgnonlin.append(nonlin)
#
#     print("Original Bert fine",  file = f)
#     print("gender_acc lin qnli:", np.mean(avglin), file = f)
#     print("gender_acc std qnli:", np.std(avglin), file = f)
#     print("gender_acc nonlin qnli:", np.mean(avgnonlin), file = f)
#     print("gender_acc std qnli:", np.std(avgnonlin), file = f)
#
#
#     f.close()
#

#


#
# #     word_pair=[[[" man "],[" woman "]]]
# #     word_correlation(word_pair, modelpath="results/5Epochs_mae/gender_con_0/model", dim = 0)
#     print("race_acc:", race_acc)
  #  print("acc:", acc)
 #   print(evaluate_model_accuracy("results/gender_con_0/model", dataset,dataset_length))


#     evaluate_model_accuracy("race_modelmrpc/understandable/model", dataset)
#     evaluate_model_accuracy("gender_contrastive/understandable/model", dataset)
#     evaluate_model_accuracy("race_contrastive/understandable/model", dataset)
#     evaluate_model_accuracy("race_modelmrpc/normal/model", dataset)
#     evaluate_model_accuracy("normal_5_epochs/normal/model", dataset)

#     evaluate_model_set("results/gender_con_",1,"results/gender_con_mae.txt",delete_dim=False)
#     evaluate_model_set("results/gender_con_",1,"results/gender_con_mae_del.txt",delete_dim=True)
#     evaluate_model_set("results/race_con_",1,"results/race_con_mae.txt",delete_dim=False)
#     evaluate_model_set("results/race_con_",1,"results/race_con_mae_del.txt",delete_dim=True)s
#     evaluate_model_set("results/race_con_",5,"results/race_con_del.txt",delete_dim=True)
  #  evaluate_model_set("results/Genderlarge/cola_gender_large_",5,"results/Genderlargecola_gender_large.txt",delete_dim=False)
  #  evaluate_model_set("results/SimpleLoss/sst2_gender_",5,"results/SimpleLoss/sst2_gender_.txt",delete_dim=False)
  #  evaluate_model_set("results/without/qnli_normal_",5,"results/without/qnli_normal_.txt",delete_dim=False)
   # evaluate_model_set("results/mrpc_genderlarge_con_",3,"results/mrpc_genderlarge_con_.txt",delete_dim=False)



  #  evaluate_model_bias("results/pre_gender_/model","results/pre_gender.txt")
#     evaluate_model_bias("race_modelmrpc/understandable/model","results/race_bert_finetuned_understandable_deleted_dim.txt", True)
    #plot_history("modelmrpc")

ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model
ERROR: Could not find a version that satisfies the requirement Understandable_Embedder (from versions: none)
ERROR: No matching distribution found for Understandable_Embedder
ERROR: Invalid requirement: 'DataLoader,'
ERROR: Invalid requirement: 'BiasMetrics,'
  Using cached data-0.4.tar.gz (7.0 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement import (from versions: none)
ERROR: No matching distribution found for import


ModuleNotFoundError: ignored

**PRE_TRAIN**

In [17]:
!pip install model
!pip install Understandable_Embedder
!pip install data import find_all_occurence_and_replace,load_pretrain_ds,get_understanding_set
import os
import pickle
from transformers import DataCollatorForLanguageModeling

if __name__ == "__main__":
    batch_size = 4


    from transformers import BertTokenizer
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    import tensorflow_datasets as tfds

    epochs = 1

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_hidden_states=True)

    pre_training_set = load_pretrain_ds( batch_size = batch_size)


    def train_understandable(train_dataset, understanding_dataset,only_dense = False, save_path = "model",debias_freq = 10):
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        model = Understandable_Embedder(batch_size, train_only_dense = only_dense, debias_freq = debias_freq)

        model.compile(optimizer=optimizer, loss=loss,metrics=["sparse_categorical_accuracy"])


        history = model.fit_pretrain(train_dataset,understanding_dataset, epochs=epochs, steps_per_epoch=int(5000), tokenizer = tokenizer) #3600 datapoints for cola
        path = "results/" + save_path
        os.makedirs(path,exist_ok=True)
        with open(path + "/history.txt", "wb") as fp:
            pickle.dump(history, fp)
        model.save_weights(path +"/model")

        return history


#     definition_pairs = [[[" good ", " positive "],[" bad ", " negative "]],
#                         [[" women ", " girl "],[" men ", " boy "]],
#                         [[" asian ", "  Asian "],[" caucasian "," Caucasian "],[" african "," African "],[" european "," European "],[" american ", " American "]],
#                         [[" Muslim ", " muslims "],[" Jew ", " jews "], [" Christian ", " christians "]]]
#
#     tokenized_definition_train_all = get_understanding_set(definition_pairs,tokenizer)


#     definition_pairs = [ [[" asian ", "  Asian "],[" caucasian "," Caucasian "],[" african "," African "],[" european "," European "],[" american ", " American "]]]
#
#     tokenized_definition_train_race = get_understanding_set(definition_pairs,tokenizer)
#
#     definition_pairs = [[[" women ", " girl "],[" men ", " boy "]]]
#
#     tokenized_definition_train_gender = get_understanding_set(definition_pairs,tokenizer)

    definition_pairs = [[[" women ", " girl ", " female ", " she ", " actress ", " heroine ", " queen "," sister ", " mother ", " lady ", " her " ],[" men ", " boy ", " male ", " he ", " actor ", " hero ", " king ", " brother ", " father ", " gentleman ", " him "]]]

    tokenized_definition_train_gender_large = get_understanding_set(definition_pairs,tokenizer)


    for i in range(7):
        train_understandable(pre_training_set,tokenized_definition_train_gender_large, save_path = ("pre_gender_5000/train" + str(2**i)), debias_freq = 2**i)



ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model
ERROR: Could not find a version that satisfies the requirement Understandable_Embedder (from versions: none)
ERROR: No matching distribution found for Understandable_Embedder
ERROR: Invalid requirement: 'find_all_occurence_and_replace,load_pretrain_ds,get_understanding_set'


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...:   0%|          | 0/5622 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/multi_news/1.0.0.incomplete8JJW1Q/multi_news-test.tfrecord*...:   0%|     …

Dataset multi_news downloaded and prepared to /root/tensorflow_datasets/multi_news/1.0.0. Subsequent calls will reuse this data.
found  20012 of word [[' women ', ' girl ', ' female ', ' she ', ' actress ', ' heroine ', ' queen ', ' sister ', ' mother ', ' lady ', ' her '], [' men ', ' boy ', ' male ', ' he ', ' actor ', ' hero ', ' king ', ' brother ', ' father ', ' gentleman ', ' him ']]


All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


AttributeError: ignored